In [101]:
import pandas as pd
import psycopg2
from typing import Dict, Set
import os
import glob
import csv
import numpy as np


# dir = "C:/Users/javi2/Documents/CD_aplicada_1/COBI/etl/data/globcolour/processed/aerosol_optical_thickness_over_water/"

# # read in the CSV files
# df1 = pd.read_csv(dir+'L3m_20170101__GLOB_100_AVW-MODVIR_T865_DAY_00_clean.csv')
# df2 = pd.read_csv(dir+'L3m_20170102__GLOB_100_AVW-MODVIR_T865_DAY_00_clean.csv')
# df3 = pd.read_csv(dir+'L3m_20170103__GLOB_100_AVW-MODVIR_T865_DAY_00_clean.csv')

# # merge the dataframes
# merged_df = pd.concat([df1, df2, df3])

# merged_df = merged_df.dropna(subset = ["T865_mean"]).iloc[:, [1,2,3,5,6]]
# merged_df.to_csv("merged_data.csv")
# merged_df["variable"] = "aerosol_optical_thickness_over_water"
# merged_df


def read_variable_dict(filename: str) -> Dict[str, str]:
    """
    Reads the variable dictionary from a CSV file and returns it as a dictionary.
    The CSV file should have a "variable" column and a "file_format" column.
    """
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        variable_dict = {row['variable']: row['file_format'] for row in reader}
    return variable_dict




In [117]:
for variable in read_variable_dict("variable_dict.csv").keys():
    
        try:
            # Set the directory where the CSV files are located
            dir_path = "C:/Users/javi2/Documents/CD_aplicada_1/COBI/etl/data/globcolour/processed/"+variable+"/"

            # Use glob to find all CSV files in the directory
            all_files = glob.glob(os.path.join(dir_path, "*.csv"))

            # Concatenate all CSV files into a single Pandas DataFrame
            df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)


            # create a dictionary to map the old column names to new column names
            new_column_names = {}
            for column in df.columns:
                if "mean" in column:
                    new_column_names[column] = "mean"
                if "error" in column:
                    new_column_names[column] = "error"

            # rename the columns using the dictionary created above
            df.rename(columns=new_column_names, inplace=True)

            if "mean" not in df.columns:
                df["mean"] = np.nan

            if "error" not in df.columns:
                df["error"] = np.nan








            # Display the concatenated DataFrame
            df = df.dropna(subset = ["mean"])[["lat", "lon", "mean", "error", "date"]]


            df.to_csv("merged_data.csv")
            df["variable"] = variable



            # set up a connection to the database
            conn = psycopg2.connect(
                host="localhost",
                database="cobi",
                user="postgres",
                password="admin"
            )

            # create a database cursor
            cur = conn.cursor()

            # Rename the DataFrame columns to match the PostgreSQL table columns
            df.columns = ["lat", "lon", "mean", "error", "date", "variable"]

            # Iterate over the DataFrame rows and insert them into the PostgreSQL table
            for i, row in df.iterrows():
                cur.execute("INSERT INTO globcolour (lat, lon, mean, error, date, variable) VALUES (%s, %s, %s, %s, %s, %s)",
                            (row["lat"], row["lon"], row["mean"], row["error"], row["date"], row["variable"]))

            # Commit the changes and close the database connection
            conn.commit()
            cur.close()
            conn.close()

        except Exception as e:
            print(variable + " error:", e)

secchi_disk_depth error: No objects to concatenate
